In [12]:
import os
import shutil


# 폴더정리
if not os.path.exists('./data/cifar-10-batches-py'):
    os.mkdir('./data/cifar-10-batches-py')

    for file in os.listdir('./data'):
        path = os.path.join('./data', file)
        if os.path.isfile(path):
            shutil.move(path, './data/cifar-10-batches-py')

print("✅ 정리 완료")

✅ 정리 완료


In [13]:
# torch & torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 데이터 전처리 설정
# 이미지를 텐서로 바꾸고, 정규화해서 값 범위를 맞춰주는 작업
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 학습 데이터셋 불러오기
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

# 테스트 데이터셋도 같은 방식으로 불러옴
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# 클래스 이름 (정답 라벨에 해당하는 것들)
classes = ['plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

In [14]:
# CNN(합성곱 신경망)을 처음 만들어봤는데, 너무 복잡하게 하지 않고 심플하게 구성함
# conv → relu → pooling → conv → relu → pooling → fc 세 개 이런 순서로 구성했음

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)      # RGB 3채널 → 6채널
        self.pool = nn.MaxPool2d(2, 2)       # 2x2 max pooling
        self.conv2 = nn.Conv2d(6, 16, 5)     # 6채널 → 16채널

        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 평탄화 후 완전연결층
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)          # 최종 클래스 개수 10개

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # conv1 → relu → pool
        x = self.pool(torch.relu(self.conv2(x)))  # conv2 → relu → pool
        x = x.view(-1, 16 * 5 * 5)                # flatten
        x = torch.relu(self.fc1(x))              # fc1 → relu
        x = torch.relu(self.fc2(x))              # fc2 → relu
        x = self.fc3(x)                          # fc3 (softmax 없음 → loss에서 처리)
        return x

net = SimpleCNN()

In [16]:
# CrossEntropyLoss를 사용함 (내부적으로 softmax 포함됨)
criterion = nn.CrossEntropyLoss()

# optimizer는 Adam을 사용
optimizer = optim.Adam(net.parameters(), lr=0.001)

# GPU가 가능하면 쓰고, 안 되면 CPU라도 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [17]:
num_epochs = 5  # 너무 오래 걸리지 않게 5번만 돌림

for epoch in range(num_epochs):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()       # gradient 초기화
        outputs = net(inputs)       # forward pass
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()             # backward pass
        optimizer.step()            # weight 업데이트

        running_loss += loss.item()

        # 100번마다 평균 손실 출력 (손실값이 줄어드는지 확인용)
        if i % 100 == 99:
            print(f'[Epoch {epoch+1}, Step {i+1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('✅ 학습 완료!')

[Epoch 1, Step 100] loss: 2.126
[Epoch 1, Step 200] loss: 1.881
[Epoch 1, Step 300] loss: 1.752
[Epoch 1, Step 400] loss: 1.673
[Epoch 1, Step 500] loss: 1.623
[Epoch 1, Step 600] loss: 1.618
[Epoch 1, Step 700] loss: 1.535
[Epoch 1, Step 800] loss: 1.561
[Epoch 1, Step 900] loss: 1.531
[Epoch 1, Step 1000] loss: 1.512
[Epoch 1, Step 1100] loss: 1.460
[Epoch 1, Step 1200] loss: 1.455
[Epoch 1, Step 1300] loss: 1.433
[Epoch 1, Step 1400] loss: 1.414
[Epoch 1, Step 1500] loss: 1.391
[Epoch 2, Step 100] loss: 1.392
[Epoch 2, Step 200] loss: 1.346
[Epoch 2, Step 300] loss: 1.327
[Epoch 2, Step 400] loss: 1.312
[Epoch 2, Step 500] loss: 1.321
[Epoch 2, Step 600] loss: 1.282
[Epoch 2, Step 700] loss: 1.281
[Epoch 2, Step 800] loss: 1.284
[Epoch 2, Step 900] loss: 1.300
[Epoch 2, Step 1000] loss: 1.270
[Epoch 2, Step 1100] loss: 1.287
[Epoch 2, Step 1200] loss: 1.240
[Epoch 2, Step 1300] loss: 1.266
[Epoch 2, Step 1400] loss: 1.264
[Epoch 2, Step 1500] loss: 1.219
[Epoch 3, Step 100] loss: 1.

In [18]:
correct = 0
total = 0
net.eval()  # 평가 모드로 전환

# 예측만 할 거니까 gradient 계산은 안 해도 됨
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)  # 가장 확률 높은 클래스 뽑음
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'🧪 테스트 데이터 정확도: {100 * correct / total:.2f}%')

🧪 테스트 데이터 정확도: 61.14%
